In [4]:
import numpy as np
import pandas as pd
import folium

In [145]:
path_moscow = "./data/moskva.geojson"
data_moscow = pd.read_json(path_moscow)

In [146]:
data_moscow = pd.json_normalize(data_moscow.features)

In [147]:
%reload_ext autoreload
%autoreload 2
from transport import transport_to_groups, transport_groups
from health_status import injury_to_severity_driver, injury_to_severity_pedestrian
from violations import violation_encoding_pedestrian, violation_encoding_driver
from nearby import property2cat

In [148]:
data_moscow

,type,geometry.type,geometry.coordinates,properties.id,properties.tags,properties.light,properties.point.lat,properties.point.long,properties.nearby,properties.region,...,properties.datetime,properties.severity,properties.vehicles,properties.dead_count,properties.participants,properties.injured_count,properties.parent_region,properties.road_conditions,properties.participants_count,properties.participant_categories
0,Feature,Point,"[37.706516, 55.616777]",1532855,[Дорожно-транспортные происшествия],Светлое время суток,55.616777,37.706516,"[Многоквартирные жилые дома, Внутридворовая те...",Орехово-Борисово Северное,...,2020-03-24 13:06:00,Легкий,"[{'year': 2012, 'brand': 'CHEVROLET', 'color':...",0,"[{'role': 'Пешеход', 'gender': 'Женский', 'vio...",1,Москва,[Сухое],2,"[Все участники, Пешеходы]"
1,Feature,Point,"[37.489332, 55.841157]",1530311,[Дорожно-транспортные происшествия],Светлое время суток,55.841157,37.489332,"[Многоквартирные жилые дома, Выезд с прилегающ...",Головинский,...,2018-09-15 11:00:00,Легкий,"[{'year': 2008, 'brand': 'MITSUBISHI', 'color'...",0,[],1,Москва,[Сухое],2,[Все участники]
2,Feature,Point,"[37.658944, 55.713332]",1529756,[Дорожно-транспортные происшествия],"В темное время суток, освещение включено",55.713332,37.658944,"[Многоквартирные жилые дома, Нерегулируемый пе...",Даниловский,...,2016-04-14 21:25:00,Легкий,[],0,"[{'role': 'Пешеход', 'gender': 'Мужской', 'vio...",1,Москва,[Сухое],2,"[Все участники, Пешеходы]"
3,Feature,Point,"[37.414176, 55.655243]",1529209,[Дорожно-транспортные происшествия],Светлое время суток,55.655243,37.414176,"[Многоквартирные жилые дома, Административные ...",Солнцево,...,2019-06-20 18:15:00,Легкий,"[{'year': 2019, 'brand': 'KIA', 'color': 'Белы...",0,[],1,Москва,[Сухое],2,[Все участники]
4,Feature,Point,"[37.789315, 55.756493]",1529849,[Дорожно-транспортные происшествия],Светлое время суток,55.756493,37.789315,"[Многоквартирные жилые дома, Выезд с прилегающ...",Перово,...,2016-04-04 11:50:00,Легкий,"[{'year': 2008, 'brand': 'MAZDA', 'color': 'Не...",0,"[{'role': 'Пешеход', 'gender': 'Мужской', 'vio...",1,Москва,[Сухое],2,"[Все участники, Пешеходы]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56443,Feature,Point,"[37.485758, 55.661509]",1527860,"[Дорожно-транспортные происшествия, ДТП и пост...",Светлое время суток,55.661509,37.485758,[],Тропарево-Никулино,...,2018-02-06 10:30:00,Легкий,"[{'year': 2009, 'brand': 'FORD', 'color': 'Чер...",0,[],1,Москва,"[Обработанное противогололедными материалами, ...",3,"[Все участники, Дети]"
56444,Feature,Point,"[37.45617, 55.70849]",1527874,[Дорожно-транспортные происшествия],Светлое время суток,55.708490,37.456170,[Остановка общественного транспорта],Очаково-Матвеевское,...,2018-02-26 17:09:00,Тяжёлый,"[{'year': 2002, 'brand': 'SKODA', 'color': 'Се...",0,"[{'role': 'Пешеход', 'gender': 'Женский', 'vio...",1,Москва,[Обработанное противогололедными материалами],2,"[Все участники, Пешеходы]"
56445,Feature,Point,"[37.569811, 55.754008999999996]",1527916,[Дорожно-транспортные происшествия],Светлое время суток,55.754009,37.569811,"[Многоквартирные жилые дома, Административные ...",Пресненский,...,2018-02-20 07:58:00,Легкий,"[{'year': 2014, 'brand': 'KIA', 'color': 'Желт...",0,[],2,Москва,[Обработанное противогололедными материалами],4,[Все участники]
56446,Feature,Point,"[37.395948, 55.376575]",1527975,[Дорожно-транспортные происшествия],"В темное время суток, освещение включено",55.376575,37.395948,[Регулируемый перекресток],Щаповское поселение,...,2018-02-17 07:20:00,Тяжёлый,"[{'year': 2014, 'brand': 'SCANIA', 'color': 'Б...",0,[],1,Москва,[Мокрое],2,[Все участники]


In [149]:
data_moscow['vehicles'] = data_moscow['properties.vehicles'].apply(lambda x: [transport_to_groups[car['category']] for car in x]) # list of vehicles which were involved in an accident

In [150]:
data_moscow['vehicle_violations'] = data_moscow['properties.vehicles'].apply(lambda x: [violation_encoding_driver[v] for car in x for person in car['participants'] for v in person['violations'] if person['violations']]) # list of violations made by participants

In [151]:
data_moscow['health_status_drivers'] = data_moscow['properties.vehicles'].apply(lambda x: [injury_to_severity_driver[person['health_status']] for car in x for person in car['participants'] if person['health_status']]) # list of injuries/health issues in an accident

In [152]:
data_moscow['experience'] = data_moscow['properties.vehicles'].apply(lambda x: [participant['years_of_driving_experience'] if participant['years_of_driving_experience'] else 0 for car in x for participant in car['participants'] if participant['role'] == 'Водитель']) # all driver participants driving experience

In [153]:
data_moscow['nearby_objects'] = data_moscow['properties.nearby'].apply(lambda x: [property2cat[obj] for obj in x])

In [154]:
data_moscow['pedestrian_violations'] = data_moscow['properties.participants'].apply(lambda x: [violation_encoding_pedestrian[v] for person in x for v in person['violations']])

In [155]:
data_moscow['health_status_pedestrians'] = data_moscow['properties.participants'].apply(lambda x: [injury_to_severity_pedestrian[person['health_status']] for person in x if person['health_status']])

In [156]:
def get_gender_of_driver(x):
    hs = []
    for car in x:
        driver_g = car['participants'][0]['gender']
        hs.append(driver_g)
    return hs
    
def get_p(x):
    x = np.array(x)
    if len(x):
        p = len(x[x=='Женский']) / len(x)
        return np.float32(p)
    return np.NAN

data_moscow['Driver_gender'] = data_moscow['properties.vehicles'].apply(lambda x: get_gender_of_driver(x))
data_moscow['w_percent'] = data_moscow['Driver_gender'].apply(lambda x: get_p(x))
del data_moscow['Driver_gender']

In [157]:
data_moscow

,type,geometry.type,geometry.coordinates,properties.id,properties.tags,properties.light,properties.point.lat,properties.point.long,properties.nearby,properties.region,...,properties.participants_count,properties.participant_categories,vehicles,vehicle_violations,health_status_drivers,experience,nearby_objects,pedestrian_violations,health_status_pedestrians,w_percent
0,Feature,Point,"[37.706516, 55.616777]",1532855,[Дорожно-транспортные происшествия],Светлое время суток,55.616777,37.706516,"[Многоквартирные жилые дома, Внутридворовая те...",Орехово-Борисово Северное,...,2,"[Все участники, Пешеходы]",[Легковые автомобили],[Водитель Другое],[Водитель Не пострадал],[28],"[Иные объекты, Иные объекты, Иные объекты]",[],[Ранения],0.000000
1,Feature,Point,"[37.489332, 55.841157]",1530311,[Дорожно-транспортные происшествия],Светлое время суток,55.841157,37.489332,"[Многоквартирные жилые дома, Выезд с прилегающ...",Головинский,...,2,[Все участники],"[Легковые автомобили, Грузовые автомобили]",[Водитель Другое],"[Водитель Ранения, Водитель Не пострадал]","[20, 6]","[Иные объекты, Места скопления автотранспорта,...",[],[],0.500000
2,Feature,Point,"[37.658944, 55.713332]",1529756,[Дорожно-транспортные происшествия],"В темное время суток, освещение включено",55.713332,37.658944,"[Многоквартирные жилые дома, Нерегулируемый пе...",Даниловский,...,2,"[Все участники, Пешеходы]",[],[],[],[],"[Иные объекты, Нерегулируемые пешеходные перех...",[],[Ранения],NaN
3,Feature,Point,"[37.414176, 55.655243]",1529209,[Дорожно-транспортные происшествия],Светлое время суток,55.655243,37.414176,"[Многоквартирные жилые дома, Административные ...",Солнцево,...,2,[Все участники],"[Легковые автомобили, Легковые автомобили]","[Водитель Нарушения при движении, Водитель Дру...","[Водитель Не пострадал, Водитель Ранения]","[12, 1]","[Иные объекты, Иные объекты, Места скопления л...",[],[],0.500000
4,Feature,Point,"[37.789315, 55.756493]",1529849,[Дорожно-транспортные происшествия],Светлое время суток,55.756493,37.789315,"[Многоквартирные жилые дома, Выезд с прилегающ...",Перово,...,2,"[Все участники, Пешеходы]",[Легковые автомобили],"[Водитель Нарушения при движении, Водитель Дру...",[Водитель Не пострадал],[15],"[Иные объекты, Места скопления автотранспорта,...",[],[Ранения],0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56443,Feature,Point,"[37.485758, 55.661509]",1527860,"[Дорожно-транспортные происшествия, ДТП и пост...",Светлое время суток,55.661509,37.485758,[],Тропарево-Никулино,...,3,"[Все участники, Дети]","[Легковые автомобили, Легковые автомобили]",[Водитель Нарушения при движении],"[Водитель Не пострадал, Водитель Ранения, Води...","[8, 10]",[],[],[],1.000000
56444,Feature,Point,"[37.45617, 55.70849]",1527874,[Дорожно-транспортные происшествия],Светлое время суток,55.708490,37.456170,[Остановка общественного транспорта],Очаково-Матвеевское,...,2,"[Все участники, Пешеходы]",[Легковые автомобили],[],[Водитель Не пострадал],[11],[Места скопления людей],[Пешеходы],[Ранения],0.000000
56445,Feature,Point,"[37.569811, 55.754008999999996]",1527916,[Дорожно-транспортные происшествия],Светлое время суток,55.754009,37.569811,"[Многоквартирные жилые дома, Административные ...",Пресненский,...,4,[Все участники],"[Легковые автомобили, Легковые автомобили, Лег...",[Водитель Нарушения при движении],"[Водитель Ранения, Водитель Ранения, Водитель ...","[0, 22, 9]","[Иные объекты, Иные объекты, Регулируемые пере...",[],[],0.333333
56446,Feature,Point,"[37.395948, 55.376575]",1527975,[Дорожно-транспортные происшествия],"В темное время суток, освещение включено",55.376575,37.395948,[Регулируемый перекресток],Щаповское поселение,...,2,[Все участники],"[Грузовые автомобили, Легковые автомобили]",[Водитель Нарушения при движении],"[Водитель Не пострадал, Водитель Ранения]","[33, 33]",[Регулируемые перекрестки],[],[],0.000000


In [158]:
del data_moscow['type']
del data_moscow['geometry.type']
del data_moscow['properties.id']
del data_moscow['properties.tags']
del data_moscow['properties.datetime']
del data_moscow['geometry.coordinates']
del data_moscow['properties.address']
del data_moscow['properties.parent_region']
del data_moscow['properties.participant_categories']
del data_moscow['properties.vehicles']
del data_moscow['properties.participants']
del data_moscow['properties.nearby']

In [159]:
data_moscow

,properties.light,properties.point.lat,properties.point.long,properties.region,properties.weather,properties.category,properties.severity,properties.dead_count,properties.injured_count,properties.road_conditions,properties.participants_count,vehicles,vehicle_violations,health_status_drivers,experience,nearby_objects,pedestrian_violations,health_status_pedestrians,w_percent
0,Светлое время суток,55.616777,37.706516,Орехово-Борисово Северное,[Ясно],Наезд на пешехода,Легкий,0,1,[Сухое],2,[Легковые автомобили],[Водитель Другое],[Водитель Не пострадал],[28],"[Иные объекты, Иные объекты, Иные объекты]",[],[Ранения],0.000000
1,Светлое время суток,55.841157,37.489332,Головинский,[Пасмурно],Столкновение,Легкий,0,1,[Сухое],2,"[Легковые автомобили, Грузовые автомобили]",[Водитель Другое],"[Водитель Ранения, Водитель Не пострадал]","[20, 6]","[Иные объекты, Места скопления автотранспорта,...",[],[],0.500000
2,"В темное время суток, освещение включено",55.713332,37.658944,Даниловский,[Пасмурно],Наезд на пешехода,Легкий,0,1,[Сухое],2,[],[],[],[],"[Иные объекты, Нерегулируемые пешеходные перех...",[],[Ранения],NaN
3,Светлое время суток,55.655243,37.414176,Солнцево,[Ясно],Столкновение,Легкий,0,1,[Сухое],2,"[Легковые автомобили, Легковые автомобили]","[Водитель Нарушения при движении, Водитель Дру...","[Водитель Не пострадал, Водитель Ранения]","[12, 1]","[Иные объекты, Иные объекты, Места скопления л...",[],[],0.500000
4,Светлое время суток,55.756493,37.789315,Перово,[Ясно],Наезд на пешехода,Легкий,0,1,[Сухое],2,[Легковые автомобили],"[Водитель Нарушения при движении, Водитель Дру...",[Водитель Не пострадал],[15],"[Иные объекты, Места скопления автотранспорта,...",[],[Ранения],0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56443,Светлое время суток,55.661509,37.485758,Тропарево-Никулино,[Пасмурно],Столкновение,Легкий,0,1,"[Обработанное противогололедными материалами, ...",3,"[Легковые автомобили, Легковые автомобили]",[Водитель Нарушения при движении],"[Водитель Не пострадал, Водитель Ранения, Води...","[8, 10]",[],[],[],1.000000
56444,Светлое время суток,55.708490,37.456170,Очаково-Матвеевское,[Ясно],Наезд на пешехода,Тяжёлый,0,1,[Обработанное противогололедными материалами],2,[Легковые автомобили],[],[Водитель Не пострадал],[11],[Места скопления людей],[Пешеходы],[Ранения],0.000000
56445,Светлое время суток,55.754009,37.569811,Пресненский,[Пасмурно],Наезд на стоящее ТС,Легкий,0,2,[Обработанное противогололедными материалами],4,"[Легковые автомобили, Легковые автомобили, Лег...",[Водитель Нарушения при движении],"[Водитель Ранения, Водитель Ранения, Водитель ...","[0, 22, 9]","[Иные объекты, Иные объекты, Регулируемые пере...",[],[],0.333333
56446,"В темное время суток, освещение включено",55.376575,37.395948,Щаповское поселение,[Пасмурно],Столкновение,Тяжёлый,0,1,[Мокрое],2,"[Грузовые автомобили, Легковые автомобили]",[Водитель Нарушения при движении],"[Водитель Не пострадал, Водитель Ранения]","[33, 33]",[Регулируемые перекрестки],[],[],0.000000


In [160]:
def cat2binary(df):
    """
    unique: values to be encoded
    df: pd.DataFrame
    return:
          matrix -- encoding
          cat2idx.keys() -- keys of dict
    """

    df_size = df.shape[0]

    unique = set()
    for i in range(df_size):
        row = df.iloc[i]
        for item in row:
            unique.add(item)
    
    unique_size = len(unique)
    cat2idx = {key: idx for idx, key in enumerate(unique)}
    matrix = np.zeros((df_size, unique_size), dtype=np.int8) 

    for i in range(df_size):
        arr = df.iloc[i]
        for word in arr:
            idx = cat2idx[word]
            matrix[i, idx] = 1
    return matrix, cat2idx.keys()

In [161]:
%%time
columns2encode = ['properties.weather', 'properties.road_conditions', 'vehicles', 'vehicle_violations', 'health_status_drivers', 'nearby_objects','pedestrian_violations', 'health_status_pedestrians']
data2merge = [data_moscow] # list of dataframes
for c in columns2encode:
    tmp_m, tmp_d = cat2binary(data_moscow[c])
    tmp_df = pd.DataFrame(tmp_m, columns=tmp_d)
    data2merge.append(tmp_df)

CPU times: user 4.89 s, sys: 41.3 ms, total: 4.93 s
Wall time: 5.13 s


In [162]:
clean_data = pd.concat(data2merge, axis=1)

In [163]:
clean_data.drop(columns2encode, axis=1, inplace=True)

In [164]:
clean_data.rename(columns={"properties.light": 'Условия освещения', "properties.point.lat": 'Широта', "properties.point.long": 'Долгота',
"properties.region": 'Регион Москвы', 'properties.severity': 'Тяжесть травм', 'properties.dead_count': 'Количество погибших', 'properties.injured_count': 'Количество пострадавших', 'properties.participants_count': 'Количество участников', 'experience': 'Водительский стаж', 'w_percent': '% женщин в авариях', 'properties.category': 'Тип аварии'}, inplace=True)

In [167]:
clean_data.to_csv("clean_data_MSK.csv", sep=':', index=False)

In [168]:
test = pd.read_csv("clean_data_MSK.csv", sep=':')